In [ ]:
import xarray as xr
import sys
import numpy as np
from pyrte_rrtmgp.utils import extract_gas_names

rte_rrtmgp_dir = "../../rrtmgp-data"
clear_sky_example_files = f"{rte_rrtmgp_dir}/examples/rfmip-clear-sky/inputs"

rfmip = xr.load_dataset(
    f"{clear_sky_example_files}/multiple_input4MIPs_radiation_RFMIP_UColorado-RFMIP-1-2_none.nc"
)

rfmip = rfmip.sel(expt=0)  # only one experiment
kdist_lw = xr.load_dataset(f"{rte_rrtmgp_dir}/rrtmgp-gas-lw-g256.nc")

# RRTMGP won't run with pressure less than its minimum. so we add a small value to the minimum pressure
press_min = rfmip["pres_level"].min()
min_index = rfmip["pres_level"].argmin()
rfmip["pres_level"][min_index] = press_min + sys.float_info.epsilon

kdist_gas_names = extract_gas_names(kdist_lw["gas_names"].values)
rfmip_vars = list(rfmip.keys())

gas_names = {n: n + "_GM" for n in kdist_gas_names if n + "_GM" in rfmip_vars}

# Create a dict that maps the gas names in the kdist gas names to the gas names in the rfmip dataset
gas_names.update(
    {
        "co": "carbon_monoxide_GM",
        "ch4": "methane_GM",
        "o2": "oxygen_GM",
        "n2o": "nitrous_oxide_GM",
        "n2": "nitrogen_GM",
        "co2": "carbon_dioxide_GM",
        "ccl4": "carbon_tetrachloride_GM",
        "cfc22": "hcfc22_GM",
        "h2o": "water_vapor",
        "o3": "ozone",
    }
)

In [ ]:
from pyrte_rrtmgp.rrtmgp import interpolation
from pyrte_rrtmgp.utils import rfmip_2_col_gas, flavors_from_kdist

neta = len(kdist_lw["mixing_fraction"])
press_ref = kdist_lw["press_ref"].values
temp_ref = kdist_lw["temp_ref"].values
tlay = rfmip["temp_layer"].values
play = rfmip["pres_layer"].values
press_ref_trop = kdist_lw["press_ref_trop"].values.item()

# start on 1 as we ignore dry air that is 0
vmr_idx = [i for i, g in enumerate(kdist_gas_names, 1) if g in gas_names]
vmr_ref = kdist_lw["vmr_ref"].sel(absorber_ext=vmr_idx).values.transpose(2, 1, 0)

col_gas = rfmip_2_col_gas(rfmip, gas_names.values(), dry_air=False)
flavor = flavors_from_kdist(kdist_lw)

jtemp, fmajor, fminor, col_mix, tropo, jeta, jpress = interpolation(
    neta=neta,
    flavor=flavor,
    press_ref=press_ref,
    temp_ref=temp_ref,
    press_ref_trop=press_ref_trop,
    vmr_ref=vmr_ref,
    play=play,
    tlay=tlay,
    col_gas=col_gas,
)

In [ ]:
from pyrte_rrtmgp.rrtmgp import compute_planck_source
from pyrte_rrtmgp.utils import gpoint_flavor_from_kdist

tlay = rfmip["temp_layer"].values
tlev = rfmip["temp_level"].values
play = rfmip["pres_layer"].values
tsfc = rfmip["surface_temperature"].values
pres_layers = rfmip["pres_layer"]["layer"]
top_at_1 = pres_layers[0] < pres_layers[-1]
band_lims_gpt = kdist_lw["bnd_limits_gpt"].values.T
pfracin = kdist_lw["plank_fraction"].values.transpose(0, 2, 1, 3)
temp_ref_min = kdist_lw["temp_ref"].values.min()
temp_ref_max = kdist_lw["temp_ref"].values.max()
totplnk = kdist_lw["totplnk"].values.T
gpoint_flavor = gpoint_flavor_from_kdist(kdist_lw)

sfc_src, lay_src, lev_src, sfc_src_jac = compute_planck_source(
    tlay,
    tlev,
    tsfc,
    top_at_1,
    fmajor,
    jeta,
    tropo,
    jtemp,
    jpress,
    band_lims_gpt,
    pfracin,
    temp_ref_min,
    temp_ref_max,
    totplnk,
    gpoint_flavor,
)

In [ ]:
from pyrte_rrtmgp.rrtmgp import compute_tau_absorption
from pyrte_rrtmgp.utils import get_idx_minor, extract_gas_names

idx_h2o = list(gas_names).index("h2o") + 1
gpoint_flavor = gpoint_flavor_from_kdist(kdist_lw)
kmajor = kdist_lw["kmajor"].values
kminor_lower = kdist_lw["kminor_lower"].values
kminor_upper = kdist_lw["kminor_upper"].values
minor_limits_gpt_lower = kdist_lw["minor_limits_gpt_lower"].values.T
minor_limits_gpt_upper = kdist_lw["minor_limits_gpt_upper"].values.T
minor_scales_with_density_lower = kdist_lw["minor_scales_with_density_lower"].values
minor_scales_with_density_upper = kdist_lw["minor_scales_with_density_upper"].values
scale_by_complement_lower = kdist_lw["scale_by_complement_lower"].values
scale_by_complement_upper = kdist_lw["scale_by_complement_upper"].values

minor_gases_lower = extract_gas_names(kdist_lw["minor_gases_lower"].values)
minor_gases_upper = extract_gas_names(kdist_lw["minor_gases_upper"].values)
gas_name_list = list(gas_names.keys())
idx_minor_lower = get_idx_minor(gas_name_list, minor_gases_lower)
idx_minor_upper = get_idx_minor(gas_name_list, minor_gases_upper)

minor_scaling_gas_lower = extract_gas_names(kdist_lw["scaling_gas_lower"].values)
minor_scaling_gas_upper = extract_gas_names(kdist_lw["scaling_gas_upper"].values)
idx_minor_scaling_lower = get_idx_minor(gas_name_list, minor_scaling_gas_lower)
idx_minor_scaling_upper = get_idx_minor(gas_name_list, minor_scaling_gas_upper)

kminor_start_lower = kdist_lw["kminor_start_lower"].values
kminor_start_upper = kdist_lw["kminor_start_upper"].values


tau = compute_tau_absorption(
    idx_h2o,
    gpoint_flavor,
    band_lims_gpt,
    kmajor,
    kminor_lower,
    kminor_upper,
    minor_limits_gpt_lower,
    minor_limits_gpt_upper,
    minor_scales_with_density_lower,
    minor_scales_with_density_upper,
    scale_by_complement_lower,
    scale_by_complement_upper,
    idx_minor_lower,
    idx_minor_upper,
    idx_minor_scaling_lower,
    idx_minor_scaling_upper,
    kminor_start_lower,
    kminor_start_upper,
    tropo,
    col_mix,
    fmajor,
    fminor,
    play,
    tlay,
    col_gas,
    jeta,
    jtemp,
    jpress,
)

In [ ]:
from pyrte_rrtmgp.rte import lw_solver_noscat

nmus = 1
sfc_emis = rfmip["surface_emissivity"].values

inc_flux = np.zeros(sfc_src.shape)

lw_solver_noscat(
    top_at_1,
    nmus,
    tau,
    lay_src,
    lev_src,
    sfc_emis,
    sfc_src,
    inc_flux
)